In [ ]:
# rag_qa_app.py

import os
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
import streamlit as st
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader

# 1. Set OpenAI API key 
os.environ["OPENAI_API_KEY"] = "openai_api_key"

# 2. Load and prepare documents (create vector DB once)
def prepare_vector_store():
    
    loader = TextLoader("nba_articles.txt")
    documents = loader.load()

    # split
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs = text_splitter.split_documents(documents)

    # embedding
    embeddings = OpenAIEmbeddings()

    # Chroma for vector database storager
    vectorstore = Chroma.from_documents(docs, embeddings, persist_directory="./chroma_db")
    vectorstore.persist()
    print("Vector store created and persisted.")

# 3. Run App
def run_qa_app():
    
    embeddings = OpenAIEmbeddings()
    vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)
    retriever = vectorstore.as_retriever()

    # GPT Model
    llm = ChatOpenAI(model_name="gpt-4", temperature=0)

    # RAG application
    qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

    # Streamlit 
    st.title("NBA RAG Question Answering Demo")

    user_question = st.text_input("Enter your NBA-related question:")

    if user_question:
        with st.spinner("Retrieving and generating answer..."):
            answer = qa_chain.run(user_question)
        st.markdown(f"**Answer:** {answer}")

if __name__ == "__main__":
    # prepare_vector_store()to run only once excepts there's any updates

    run_qa_app()
